# Output from Transformer + LSTM into Random Forest ?


In [15]:
import sklearn
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [16]:
X_train = pd.read_pickle('../X_train.pkl')
X_test = pd.read_pickle('../X_test.pkl')
y_train = pd.read_pickle('../y_train.pkl')
y_test = pd.read_pickle('../y_test.pkl')

In [17]:
X_train = X_train.iloc[1:,:]
X_test = X_test.iloc[1:,:]
y_train = y_train.iloc[1:]
y_test = y_test.iloc[1:]

In [18]:
y_train = list(y_train[0])
y_test = list(y_test[0])

In [20]:
X_train['label']=y_train

In [22]:
X_test['label']=y_test

In [27]:
true = X_train[X_train.label==1]
false = X_train[X_train.label==0]

In [28]:
train = false[0:20000].append(true[0:1333])
test  = false[20000:22494].append(true[2000:2166])

In [30]:
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)

In [47]:
X_train = train.iloc[:,:-1]
X_test = test.iloc[:,:-1]
y_train = list(train.iloc[:,-1])
y_test = list(test.iloc[:,-1])

In [49]:
weights = sklearn.utils.class_weight.compute_class_weight(class_weight='balanced',classes=[0.0,1.0],y=y_train)
weights

array([0.533325  , 8.00187547])

In [8]:
weights[1]*2

15.469030711337703

In [88]:
model = RandomForestClassifier(class_weight={0.0:weights[0], 1.0:weights[1]}, oob_score=True, max_features=128, n_jobs=-1)
model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                       class_weight={0.0: 0.533325, 1.0: 8.001875468867217},
                       criterion='gini', max_depth=None, max_features=128,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=True, random_state=None, verbose=0,
                       warm_start=False)

In [89]:
model = model.fit(X=X_train,y=y_train)

In [90]:
model.score(X=X_train,y=y_train)

0.9999062485351333

### Training set

In [91]:
output = model.predict(X_train)
output_prob = model.predict_proba(X_train)

In [92]:
confusion = metrics.confusion_matrix(y_true=y_train, y_pred=output)
print('Confusion matrix: \n',confusion)

tn, fp, fn, tp = confusion.ravel()
print('\nTP:',tp)
print('FP:',fp)
print('TN:',tn)
print('FN:',fn)

## Performance measure
print('\nAccuracy: '+ str(metrics.accuracy_score(y_true=y_train, y_pred=output)))
print('Precision: '+ str(metrics.precision_score(y_true=y_train, y_pred=output)))
print('Recall: '+ str(metrics.recall_score(y_true=y_train, y_pred=output)))
print('F-measure: '+ str(metrics.f1_score(y_true=y_train, y_pred=output)))
print('Area Under the Curve: '+ str(metrics.roc_auc_score(y_true=y_train, y_score=output_prob[:,1])))
print('Precision-Recall AUC: '+ str(metrics.average_precision_score(y_true=y_train, y_score=output_prob[:,1])))
print('Matthew Correlation Coefficient: '+ str(metrics.matthews_corrcoef(y_true=y_train, y_pred=output)))
print('\n\n')

Confusion matrix: 
 [[20000     0]
 [    2  1331]]

TP: 1331
FP: 0
TN: 20000
FN: 2

Accuracy: 0.9999062485351333
Precision: 1.0
Recall: 0.9984996249062266
F-measure: 0.9992492492492493
Area Under the Curve: 1.0
Precision-Recall AUC: 1.0
Matthew Correlation Coefficient: 0.9991995721214725





### Testing set

In [93]:
output = model.predict(X_test)
output_prob = model.predict_proba(X_test)

In [94]:
confusion = metrics.confusion_matrix(y_true=y_test, y_pred=output)
print('Confusion matrix: \n',confusion)

tn, fp, fn, tp = confusion.ravel()
print('\nTP:',tp)
print('FP:',fp)
print('TN:',tn)
print('FN:',fn)

## Performance measure
print('\nAccuracy: '+ str(metrics.accuracy_score(y_true=y_test, y_pred=output)))
print('Precision: '+ str(metrics.precision_score(y_true=y_test, y_pred=output)))
print('Recall: '+ str(metrics.recall_score(y_true=y_test, y_pred=output)))
print('F-measure: '+ str(metrics.f1_score(y_true=y_test, y_pred=output)))
print('Area Under the Curve: '+ str(metrics.roc_auc_score(y_true=y_test, y_score=output_prob[:,1])))
print('Precision-Recall AUC: '+ str(metrics.average_precision_score(y_true=y_test, y_score=output_prob[:,1])))
print('Matthew Correlation Coefficient: '+ str(metrics.matthews_corrcoef(y_true=y_test, y_pred=output)))
print('\n\n')

Confusion matrix: 
 [[2494    0]
 [ 166    0]]

TP: 0
FP: 0
TN: 2494
FN: 166

Accuracy: 0.937593984962406
Precision: 0.0
Recall: 0.0
F-measure: 0.0
Area Under the Curve: 0.47614395029999707
Precision-Recall AUC: 0.061449449253679594
Matthew Correlation Coefficient: 0.0





/data/anaconda3/envs/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/data/anaconda3/envs/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
